In [2]:
import os
import argparse
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl
from dgl.dataloading import MultiLayerNeighborSampler, DataLoader


# ----------------------------
# util
# ----------------------------
def log(s: str):
    print(s, flush=True)

def ensure_dir(d: str):
    os.makedirs(d, exist_ok=True)

def set_seed(seed: int):
    import random
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

def infer_feature_cols(header_cols):
    # txId, Time step は特徴に入れない（必要なら後で足せます）
    drop = {"txId", "Time step"}
    return [c for c in header_cols if c not in drop]


# ----------------------------
# build assets
# ----------------------------
def build_tx_ids(features_path: str, out_path: str) -> np.ndarray:
    log("[A] txId を features から抽出...")
    tx = pd.read_csv(features_path, usecols=["txId"], dtype={"txId": np.int64})
    tx_ids = tx["txId"].to_numpy(np.int64)
    np.save(out_path, tx_ids)
    log(f"[A] N={len(tx_ids)}  saved: {out_path}")
    return tx_ids

def build_features_memmap(features_path: str, tx_ids: np.ndarray,
                          out_memmap_path: str, out_cols_path: str,
                          chunksize: int) -> int:
    log("[B] features を memmap に変換...")
    header = pd.read_csv(features_path, nrows=0)
    feat_cols = infer_feature_cols(list(header.columns))
    D = len(feat_cols)
    np.save(out_cols_path, np.array(feat_cols, dtype=object))
    log(f"[B] feature dim D={D}  cols saved: {out_cols_path}")

    N = len(tx_ids)
    mm = np.memmap(out_memmap_path, dtype=np.float32, mode="w+", shape=(N, D))

    reader = pd.read_csv(
        features_path,
        usecols=feat_cols,
        dtype={c: np.float32 for c in feat_cols},
        chunksize=chunksize,
    )

    offset = 0
    for chunk in reader:
        arr = chunk.to_numpy(np.float32, copy=False)
        n = arr.shape[0]
        mm[offset:offset+n, :] = arr
        offset += n
        if offset % 1_000_000 < chunksize:
            log(f"[B] written: {offset}/{N}")

    mm.flush()
    if offset != N:
        raise RuntimeError(f"[B] features 行数不一致: wrote={offset} N={N}")
    log(f"[B] memmap saved: {out_memmap_path}")
    return D

def build_labels(classes_path: str, tx_ids: np.ndarray, out_path: str) -> np.ndarray:
    """
    典型: class 1=illicit, 2=licit, 3=unknown
    → 学習用 y: illicit=1, licit=0, unknown=-1
    """
    log("[C] labels y を作成（unknown=-1）...")
    y = np.full((len(tx_ids),), -1, dtype=np.int8)

    idx = pd.Index(tx_ids)
    df = pd.read_csv(classes_path, dtype={"txId": np.int64, "class": np.int16})

    loc = idx.get_indexer(df["txId"].to_numpy(np.int64))
    ok = loc >= 0
    loc = loc[ok]
    cls = df["class"].to_numpy(np.int16)[ok]

    y_vals = np.full_like(cls, -1, dtype=np.int8)
    y_vals[cls == 1] = 1
    y_vals[cls == 2] = 0
    y_vals[cls == 3] = -1

    y[loc] = y_vals
    np.save(out_path, y)
    log(f"[C] saved: {out_path}  labeled={(y!=-1).sum()}  illicit={(y==1).sum()}  licit={(y==0).sum()}  unknown={(y==-1).sum()}")
    return y

def build_edges(edges_path: str, tx_ids: np.ndarray, out_path: str) -> np.ndarray:
    log("[D] edges を txId -> index に変換...")
    idx = pd.Index(tx_ids)

    edges = pd.read_csv(edges_path, dtype={"txId1": np.int64, "txId2": np.int64})
    src = edges["txId1"].to_numpy(np.int64)
    dst = edges["txId2"].to_numpy(np.int64)

    src_i = idx.get_indexer(src)
    dst_i = idx.get_indexer(dst)

    ok = (src_i >= 0) & (dst_i >= 0)
    src_i = src_i[ok].astype(np.int64)
    dst_i = dst_i[ok].astype(np.int64)

    edge_index = np.vstack([src_i, dst_i]).astype(np.int64)
    np.save(out_path, edge_index)
    log(f"[D] kept edges: {edge_index.shape[1]}  saved: {out_path}")
    return edge_index


# ----------------------------
# Model
# ----------------------------
class SAGE(nn.Module):
    def __init__(self, in_dim: int, hid: int, emb_dim: int, dropout: float = 0.2):
        super().__init__()
        self.sage1 = dgl.nn.SAGEConv(in_dim, hid, aggregator_type="mean")
        self.sage2 = dgl.nn.SAGEConv(hid, emb_dim, aggregator_type="mean")
        self.dropout = dropout

    def forward(self, blocks, x):
        h = self.sage1(blocks[0], x)
        h = F.relu(h)
        h = F.dropout(h, p=self.dropout, training=self.training)
        h = self.sage2(blocks[1], h)
        return h  # embedding


def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--features", default="txs_features.txt")
    ap.add_argument("--classes",  default="txs_classes.txt")
    ap.add_argument("--edges",    default="txs_edgelist.txt")
    ap.add_argument("--workdir",  default="./_prep_dgl")
    ap.add_argument("--emb_dim",  type=int, default=64)
    ap.add_argument("--hid_dim",  type=int, default=128)
    ap.add_argument("--epochs",   type=int, default=3)
    ap.add_argument("--batch",    type=int, default=2048)
    ap.add_argument("--neighbors", default="15,10")  # 2-layer
    ap.add_argument("--lr",       type=float, default=1e-3)
    ap.add_argument("--seed",     type=int, default=42)
    ap.add_argument("--chunksize",type=int, default=200000)
    ap.add_argument("--bidirected", action="store_true", help="グラフを双方向化（推奨）")
    ap.add_argument("--num_workers", type=int, default=0, help="Windowsは0推奨")
    args = ap.parse_args()

    set_seed(args.seed)
    ensure_dir(args.workdir)

    # generated files
    txids_path = os.path.join(args.workdir, "tx_ids.npy")
    cols_path  = os.path.join(args.workdir, "feature_cols.npy")
    x_mm_path  = os.path.join(args.workdir, "x_float32.memmap")
    y_path     = os.path.join(args.workdir, "y_int8.npy")
    e_path     = os.path.join(args.workdir, "edge_index.npy")
    emb_path   = os.path.join(args.workdir, f"emb_{args.emb_dim}d_float32.memmap")

    # A: tx_ids
    if os.path.exists(txids_path):
        tx_ids = np.load(txids_path)
        log(f"[A] reuse: {txids_path}  N={len(tx_ids)}")
    else:
        tx_ids = build_tx_ids(args.features, txids_path)

    # B: features memmap
    if os.path.exists(x_mm_path) and os.path.exists(cols_path):
        feat_cols = np.load(cols_path, allow_pickle=True)
        D = int(len(feat_cols))
        log(f"[B] reuse: {x_mm_path}  shape=({len(tx_ids)},{D})")
    else:
        D = build_features_memmap(args.features, tx_ids, x_mm_path, cols_path, args.chunksize)

    # C: labels
    if os.path.exists(y_path):
        y = np.load(y_path)
        log(f"[C] reuse: {y_path}")
    else:
        y = build_labels(args.classes, tx_ids, y_path)

    # D: edges
    if os.path.exists(e_path):
        edge_index = np.load(e_path)
        log(f"[D] reuse: {e_path}  E={edge_index.shape[1]}")
    else:
        edge_index = build_edges(args.edges, tx_ids, e_path)

    # graph
    N = len(tx_ids)
    src = torch.from_numpy(edge_index[0])
    dst = torch.from_numpy(edge_index[1])
    g = dgl.graph((src, dst), num_nodes=N)
    if args.bidirected:
        g = dgl.to_bidirected(g, copy_ndata=False)
        log("[INFO] graph -> bidirected")
    g = dgl.add_self_loop(g)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    log(f"[INFO] device={device}")
    log(f"[INFO] nodes={g.num_nodes()} edges={g.num_edges()}  D={D}")

    # attach features/labels
    x_mm = np.memmap(x_mm_path, dtype=np.float32, mode="r", shape=(N, D))
    g.ndata["x"] = torch.from_numpy(x_mm)                      # CPU参照
    g.ndata["y"] = torch.from_numpy(y.astype(np.int64))        # CPU

    # labeled nodes only (y != -1)
    labeled_mask = (g.ndata["y"] != -1)
    labeled_idx = torch.nonzero(labeled_mask, as_tuple=False).squeeze()
    if labeled_idx.numel() == 0:
        raise RuntimeError("学習用ラベルが0件です。txs_classes.txt の class 定義を確認してください。")

    # train/val split
    perm = labeled_idx[torch.randperm(labeled_idx.numel())]
    n_train = int(0.8 * perm.numel())
    train_idx = perm[:n_train]
    val_idx   = perm[n_train:]

    # sampler / loaders
    neigh = [int(v) for v in args.neighbors.split(",")]
    if len(neigh) != 2:
        raise ValueError("--neighbors は '15,10' のように2層分を指定してください。")
    sampler = MultiLayerNeighborSampler(neigh)

    train_loader = DataLoader(
        g, train_idx, sampler,
        batch_size=args.batch, shuffle=True, drop_last=False, num_workers=args.num_workers
    )
    val_loader = DataLoader(
        g, val_idx, sampler,
        batch_size=args.batch, shuffle=False, drop_last=False, num_workers=args.num_workers
    )

    # model
    model = SAGE(D, args.hid_dim, args.emb_dim, dropout=0.2).to(device)
    clf = nn.Linear(args.emb_dim, 2).to(device)
    opt = torch.optim.Adam(list(model.parameters()) + list(clf.parameters()), lr=args.lr, weight_decay=1e-4)

    def eval_acc():
        model.eval(); clf.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for _, _, blocks in val_loader:
                blocks = [b.to(device) for b in blocks]
                xb = blocks[0].srcdata["x"].to(device)
                yb = blocks[-1].dstdata["y"].to(device)
                z = model(blocks, xb)
                logits = clf(z)
                pred = logits.argmax(dim=-1)
                correct += int((pred == yb).sum().item())
                total += int(yb.numel())
        return correct / max(1, total)

    # ----------------------------
    # Train
    # ----------------------------
    log("[TRAIN] start")
    for epoch in range(1, args.epochs + 1):
        model.train(); clf.train()
        total_loss = 0.0
        steps = 0
        for _, _, blocks in train_loader:
            blocks = [b.to(device) for b in blocks]
            xb = blocks[0].srcdata["x"].to(device)
            yb = blocks[-1].dstdata["y"].to(device)  # 0/1
            z = model(blocks, xb)
            logits = clf(z)
            loss = F.cross_entropy(logits, yb)

            opt.zero_grad()
            loss.backward()
            opt.step()

            total_loss += float(loss.item())
            steps += 1

        acc = eval_acc()
        log(f"[TRAIN] epoch={epoch} loss={total_loss/max(1,steps):.4f} val_acc={acc:.4f}")

    # ----------------------------
    # Embedding generation (ALL nodes) with NeighborSampler
    # ----------------------------
    log("[EMB] generate embeddings for ALL nodes (neighbor sampling) ...")
    all_idx = torch.arange(N)
    full_loader = DataLoader(
        g, all_idx, sampler,  # ★ FullNeighborSamplerは使わない（巨大グラフで爆発）
        batch_size=args.batch, shuffle=False, drop_last=False, num_workers=args.num_workers
    )

    emb_mm = np.memmap(emb_path, dtype=np.float32, mode="w+", shape=(N, args.emb_dim))

    model.eval()
    with torch.no_grad():
        done = 0
        for input_nodes, output_nodes, blocks in full_loader:
            blocks = [b.to(device) for b in blocks]
            xb = blocks[0].srcdata["x"].to(device)
            z = model(blocks, xb)  # [batch, emb_dim] (dst側＝output_nodes)

            out_nids = output_nodes.cpu().numpy()
            emb_mm[out_nids, :] = z.detach().cpu().numpy().astype(np.float32, copy=False)

            done += len(out_nids)
            if done % 1_000_000 < args.batch:
                log(f"[EMB] done nodes: {done}/{N}")

    emb_mm.flush()
    log(f"[EMB] saved: {emb_path}")
    log("[DONE]")


if __name__ == "__main__":
    main()


FileNotFoundError: Cannot find DGL C++ graphbolt library at C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\dgl\graphbolt\graphbolt_pytorch_2.6.0.dll